In [3]:
__maintainer__ = "[José M. Beltrán](<jobel.open.science@gmail.com>)"
__modified_from = {"code":"matlab","credits":"Shungudzemwoyo Garaba","email":"<shungu.garaba@uni-oldenburg.de>", "dates":{"created":20110224, "modified":20140910}}
__credits__ = ["José M. Beltrán"]
__license__ = "GPL-3.0"
__status__ = []

# Notebook to calculate the Forel-Ule Index

**Reference** :
Wernand, M. R., Hommersom, a., & Van Der Woerd, H. J. (2013). MERIS-based ocean colour classification with the discrete Forel-Ule scale. Ocean Science, 9, 477–487. doi:10.5194/os-9-477-2013

**Bibtex** : 
@article{Wernand2013, 
author = {Wernand, M. R. and Hommersom, a. and {Van Der Woerd}, H. J.}, 
title = {{MERIS-based ocean colour classification with the discrete Forel-Ule scale}},
doi = {10.5194/os-9-477-2013},
issn = {18120784},
journal = {Ocean Science},
pages = {477--487},
volume = {9},
year = {2013}
}

In [4]:
# Keeping a local mathjax for Latex rendering
from IPython.external import mathjax  #mathjax.install_mathjax()

In [5]:
import os
import pandas as pd
from scipy.interpolate import interp1d
import math

In [10]:
# Set the current working directory
#os.chdir('/home/jobel/gits/jobel/FUME')
!cd

C:\Users\tams00\OneDrive - CEFAS\git\FUME


[**Notes from the colour and vision research laboratory**](http://www.cvrl.org/)

The CIE 1931 2-deg CMFs (CIE, 1932), which form the basis for most practical colorimetry, are based on the chromaticity coordinates obtained by Guild (1931) and by Wright (1928). Chromaticity coordinates, however, provide only a relative measure of the ratios of the three primaries needed to match each spectrum color, whereas CMFs specify absolute energy values. In order to reconstruct the CMFs from the Wright and Guild data, it was assumed that the CIE1924 V(l) photopic luminosity function (CIE, 1926) is a linear combination of the three CMFs (see Wyszecki & Stiles, 1982), for a description of the reconstruction and for the tabulated values. 

It has long been clear that the CIE1924 V(λ) that was used to construct the CIE 1931 2-deg CMFs **seriously underestimates sensitivity at wavelengths below 460 nm**, so that these CMFs are seriously in error at short wavelengths. The Judd and Judd-Vos modifications are attempts to overcome this problem.


In [35]:
cmf = pd.read_csv(sep = "\t", filepath_or_buffer = "data/FUI_CIE1931_JV.tsv")
fui = pd.read_csv(sep = "\t", filepath_or_buffer = "data/FUI_ATAN210.tsv", names = ["value", "atan"])
test = pd.read_csv(sep = "\t", filepath_or_buffer = "data/test.csv", header = None)
# Renaming the columns of the test dataframe
test.columns = ["wavelength", "y1", "y2", "y3", "y4"]

In [36]:
#the interpolating part 
Delta = 5 # 5nm bins
# adds +5 as it should include 825 resulting in 90 values, i.e. matching the len(cmf).
xi1 = range(380, 825 + Delta, Delta)  


In [37]:
# For linear interpolation you can use:
# interp1d(x,y)(new_x) # 'linear' it's the default
# or to be more explicit
# set_interp = interp1d(x,y, kind='linear')
#new_y = set_interp(new_x)

In [38]:
# retrieving the reflectance values for lambda 380-720nm
int_r1 = interp1d(test["wavelength"], test["y1"], kind = 'linear')(xi1)

In [39]:
cmf

,wavelength,x,y,z
0,380,0.002690,2.000000e-04,1.226000e-02
1,385,0.005311,3.955600e-04,2.422200e-02
2,390,0.010781,8.000000e-04,4.925000e-02
3,395,0.020792,1.545700e-03,9.513500e-02
4,400,0.037981,2.800000e-03,1.740900e-01
...,...,...,...,...
85,805,0.000007,2.607600e-06,5.862600e-10
86,810,0.000005,1.836500e-06,4.213800e-10
87,815,0.000003,1.295000e-06,3.031900e-10
88,820,0.000002,9.109200e-07,2.175300e-10


In [40]:
# Creating a dictionary to hold the reflectance values aas tristimulus
r = {"x": [], "y": [], "z": []}
# ------- R_RS * cmf
for i in range(0, len(cmf)):
    r["x"] = int_r1[i] * cmf["x"]
    r["y"] = int_r1[i] * cmf["y"]
    r["z"] = int_r1[i] * cmf["z"]

In [41]:
# ------ Sum
s = {"x": [], "y": [], "z": []}

s["x"] = sum(r["x"] * Delta)
s["y"] = sum(r["y"] * Delta)
s["z"] = sum(r["z"] * Delta)

sum_xyz = s["x"] + s["y"] + s["z"]

In [42]:
# ------ chromaticity
chrom = {"x": [], "y": [], "z": []}
chrom["x"] = s["x"] / sum_xyz
chrom["y"] = s["y"] / sum_xyz
chrom["z"] = s["z"] / sum_xyz

sum_chrom_y = chrom["x"] + chrom["y"] + chrom["z"]

In [43]:
# ______ chromaticity - whiteness
chrom_w = {"x": [], "y": []}
chrom_w["x"] = chrom["x"] - (1 / 3)
chrom_w["y"] = chrom["y"] - (1 / 3)

In [44]:
# ______ calculate atan2
# we use the average atan per scale
a_i = math.atan2(chrom_w["y"], chrom_w["x"]) * 180 / math.pi

if a_i < 0:
    a_i = a_i + 360
else:
    a_i = a_i

In [46]:
# ----- fui approximation

if a_i >= fui["atan"][0]:  # FUI = 1 its > Average
    fu_i = 1.0
elif math.isnan(a_i):  # FUI = NAN = 0
    fu_i = 0
elif a_i <= fui["atan"][200]:  # FUI = 21
    fu_i = 21.0
else:
    for c in range(0, 200):
        if (fui["atan"][c] > a_i) and (a_i >= fui["atan"][c + 1]):
            fu_i = fui["value"][c + 1]

In [47]:
# The calculated FUI is
fu_i

14.9